In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Tratamento matches

In [3]:
path_matches_bronze = "s3://bronze/matches/"

df_matches_bronze = spark.read.parquet(path_matches_bronze)

24/04/16 23:30:07 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
df_matches_bronze.printSchema()

root
 |-- away_score: long (nullable = true)
 |-- away_team: struct (nullable = true)
 |    |-- away_team_gender: string (nullable = true)
 |    |-- away_team_group: string (nullable = true)
 |    |-- away_team_id: long (nullable = true)
 |    |-- away_team_name: string (nullable = true)
 |    |-- country: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- managers: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- country: struct (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- dob: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- nickname: string (nullable = true)
 |-- competition: struct (nullable = true)
 |    |-- competition_id: long (nullable = true)
 |    |-- competition_nam

In [5]:
display(df_matches_bronze)

DataFrame[away_score: bigint, away_team: struct<away_team_gender:string,away_team_group:string,away_team_id:bigint,away_team_name:string,country:struct<id:bigint,name:string>,managers:array<struct<country:struct<id:bigint,name:string>,dob:string,id:bigint,name:string,nickname:string>>>, competition: struct<competition_id:bigint,competition_name:string,country_name:string>, competition_stage: struct<id:bigint,name:string>, home_score: bigint, home_team: struct<country:struct<id:bigint,name:string>,home_team_gender:string,home_team_group:string,home_team_id:bigint,home_team_name:string,managers:array<struct<country:struct<id:bigint,name:string>,dob:string,id:bigint,name:string,nickname:string>>>, kick_off: string, last_updated: string, last_updated_360: string, match_date: string, match_id: bigint, match_status: string, match_status_360: string, match_week: bigint, metadata: struct<data_version:string,shot_fidelity_version:string,xy_fidelity_version:string>, referee: struct<country:struc

In [6]:
df_matches_bronze.show()

+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+----------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+--------------+---------+
|away_score|           away_team|         competition|  competition_stage|home_score|           home_team|    kick_off|        last_updated|last_updated_360|match_date|match_id|match_status|match_status_360|match_week|     metadata|             referee|         season|             stadium|competition_id|season_id|
+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+----------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+--------------+---------+
|         2|{male, null, 322,...|{11, La Liga, Spain

In [7]:
df_matches_bronze.count()

2886

In [8]:
df_matches_duplicated = df_matches_bronze.groupBy(["match_id"]).count()

In [9]:
df_matches_duplicated.orderBy(F.col("count").desc()).show()

+--------+-----+
|match_id|count|
+--------+-----+
| 3825571|    1|
| 3754048|    1|
| 3775620|    1|
| 3825587|    1|
| 3901733|    1|
|    7546|    1|
| 3754156|    1|
| 3879693|    1|
| 3817860|    1|
| 3754073|    1|
| 3775560|    1|
| 3835324|    1|
| 3890447|    1|
| 3773523|    1|
| 3754264|    1|
| 3879668|    1|
| 3890475|    1|
| 3890344|    1|
| 3773386|    1|
| 3825652|    1|
+--------+-----+
only showing top 20 rows



In [10]:
df_matches_competition_11_season_27 = df_matches_bronze.where("season_id = 27 and competition_id = 11")

In [11]:
df_matches_competition_11_season_27.show()

+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+----------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+--------------+---------+
|away_score|           away_team|         competition|  competition_stage|home_score|           home_team|    kick_off|        last_updated|last_updated_360|match_date|match_id|match_status|match_status_360|match_week|     metadata|             referee|         season|             stadium|competition_id|season_id|
+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+----------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+--------------+---------+
|         2|{male, null, 322,...|{11, La Liga, Spain

In [12]:
df_matches_competition_11_season_27.count()

380

In [13]:
df_matches_bronze_explode = df_matches_bronze.withColumn("away_team_gender", F.col("away_team.away_team_gender"))\
                            .withColumn("away_team_group", F.col("away_team.away_team_group"))\
                            .withColumn("away_team_id", F.col("away_team.away_team_id"))\
                            .withColumn("away_team_name", F.col("away_team.away_team_name"))\
                            .withColumn("country_away_team_id", F.col("away_team.country.id"))\
                            .withColumn("country_away_team_name", F.col("away_team.country.name"))\
                            .withColumn("manager_away", F.explode("away_team.managers")) \
                            .withColumn("manager_away_country_id", F.col("manager_away.country.id")) \
                            .withColumn("manager_away_country_name", F.col("manager_away.country.name")) \
                            .withColumn("manager_away_dob", F.col("manager_away.dob")) \
                            .withColumn("manager_away_id", F.col("manager_away.id")) \
                            .withColumn("manager_away_name", F.col("manager_away.name")) \
                            .withColumn("manager_away_nickname", F.col("manager_away.nickname")) \
                            .withColumn("competition_id", F.col("competition.competition_id")) \
                            .withColumn("competition_name", F.col("competition.competition_name")) \
                            .withColumn("competition_stage_id", F.col("competition_stage.id")) \
                            .withColumn("competition_stage_name", F.col("competition_stage.name")) \
                            .withColumn("home_team_gender", F.col("home_team.home_team_gender"))\
                            .withColumn("home_team_group", F.col("home_team.home_team_group"))\
                            .withColumn("home_team_id", F.col("home_team.home_team_id"))\
                            .withColumn("home_team_name", F.col("home_team.home_team_name"))\
                            .withColumn("country_home_team_id", F.col("home_team.country.id"))\
                            .withColumn("country_home_team_name", F.col("home_team.country.name"))\
                            .withColumn("manager_home", F.explode("home_team.managers")) \
                            .withColumn("manager_home_country_id", F.col("manager_home.country.id")) \
                            .withColumn("manager_home_country_name", F.col("manager_home.country.name")) \
                            .withColumn("manager_home_dob", F.col("manager_home.dob")) \
                            .withColumn("manager_home_id", F.col("manager_home.id")) \
                            .withColumn("manager_home_name", F.col("manager_home.name")) \
                            .withColumn("manager_home_nickname", F.col("manager_home.nickname")) \
                            .withColumn("referee_country_id", F.col("referee.country.id")) \
                            .withColumn("referee_country_name", F.col("referee.country.name")) \
                            .withColumn("referee_id", F.col("referee.id")) \
                            .withColumn("referee_name", F.col("referee.name")) \
                            .withColumn("season_name", F.col("season.season_name")) \
                            .withColumn("stadium_country_id", F.col("stadium.country.id")) \
                            .withColumn("stadium_country_name", F.col("stadium.country.name")) \
                            .withColumn("stadium_id", F.col("stadium.id")) \
                            .withColumn("stadium_name", F.col("stadium.name")) \
                            .drop("away_team", "competition", "competition_stage", "home_team", "last_updated_360", "match_status_360", "metadata", "season", "stadium", "referee", "manager_away", "manager_home")           


In [14]:
df_matches_bronze_explode.printSchema()

root
 |-- away_score: long (nullable = true)
 |-- home_score: long (nullable = true)
 |-- kick_off: string (nullable = true)
 |-- last_updated: string (nullable = true)
 |-- match_date: string (nullable = true)
 |-- match_id: long (nullable = true)
 |-- match_status: string (nullable = true)
 |-- match_week: long (nullable = true)
 |-- competition_id: long (nullable = true)
 |-- season_id: integer (nullable = true)
 |-- away_team_gender: string (nullable = true)
 |-- away_team_group: string (nullable = true)
 |-- away_team_id: long (nullable = true)
 |-- away_team_name: string (nullable = true)
 |-- country_away_team_id: long (nullable = true)
 |-- country_away_team_name: string (nullable = true)
 |-- manager_away_country_id: long (nullable = true)
 |-- manager_away_country_name: string (nullable = true)
 |-- manager_away_dob: string (nullable = true)
 |-- manager_away_id: long (nullable = true)
 |-- manager_away_name: string (nullable = true)
 |-- manager_away_nickname: string (nullab

In [15]:
df_matches_bronze_explode.show()

24/04/16 23:30:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+----------+------------+--------------------+----------+--------+------------+----------+--------------+---------+----------------+---------------+------------+--------------------+--------------------+----------------------+-----------------------+-------------------------+----------------+---------------+--------------------+---------------------+----------------+--------------------+----------------------+----------------+---------------+------------+--------------------+--------------------+----------------------+-----------------------+-------------------------+----------------+---------------+--------------------+---------------------+------------------+--------------------+----------+--------------------+-----------+------------------+--------------------+----------+--------------------+
|away_score|home_score|    kick_off|        last_updated|match_date|match_id|match_status|match_week|competition_id|season_id|away_team_gender|away_team_group|away_team_id|      away

In [16]:
df_matches_bronze_explode.count()

2845

In [17]:
df_matches_not_in_explode = df_matches_bronze.join(df_matches_bronze_explode, on="match_id", how="left_anti")

In [18]:
df_matches_not_in_explode.show()

+--------+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+--------------------+----------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+--------------+---------+
|match_id|away_score|           away_team|         competition|  competition_stage|home_score|           home_team|    kick_off|        last_updated|    last_updated_360|match_date|match_status|match_status_360|match_week|     metadata|             referee|         season|             stadium|competition_id|season_id|
+--------+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+--------------------+----------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+--------------+---------+
| 3825812|         0|{male, null, 1041..

In [19]:
df_matches_bronze_explode.select("manager_away_country_id", "manager_away_country_name", "manager_away_dob", "manager_away_id", "manager_away_name", "manager_away_nickname").where("match_id = 3825848").show()


+-----------------------+-------------------------+----------------+---------------+--------------------+---------------------+
|manager_away_country_id|manager_away_country_name|manager_away_dob|manager_away_id|   manager_away_name|manager_away_nickname|
+-----------------------+-------------------------+----------------+---------------+--------------------+---------------------+
|                    214|                    Spain|      1961-03-14|            221|José Luis Mendili...| José Luis Mendilibar|
+-----------------------+-------------------------+----------------+---------------+--------------------+---------------------+



In [20]:
df_match_3825812_home_manager =  df_matches_bronze.select(F.explode("home_team.managers").alias("home_team_manager")).where("match_id = 3825812")  

In [21]:
df_match_3825812_home_manager.show()

# -> Não tem Managers

+-----------------+
|home_team_manager|
+-----------------+
+-----------------+



##  Análise managers

In [22]:
df_matches_bronze_manager_using_withcolumn_explode = df_matches_bronze \
                            .withColumn("manager_home", F.explode_outer("home_team.managers"))

In [23]:
df_matches_bronze_manager_using_withcolumn_explode_duplicated = df_matches_bronze_manager_using_withcolumn_explode.groupBy(["match_id"]).count()

df_matches_bronze_manager_using_withcolumn_explode_duplicated.orderBy(F.col("count").desc()).show()

+--------+-----+
|match_id|count|
+--------+-----+
| 3825717|    2|
| 3878541|    2|
|    9695|    2|
| 3754151|    2|
| 3825794|    2|
| 3825563|    2|
| 3825896|    2|
|    9765|    2|
| 3825571|    1|
| 3754048|    1|
| 3825587|    1|
| 3879693|    1|
| 3775620|    1|
| 3901733|    1|
|    7546|    1|
| 3817860|    1|
| 3754073|    1|
| 3835324|    1|
| 3754156|    1|
| 3775560|    1|
+--------+-----+
only showing top 20 rows



In [24]:
df_matches_bronze_manager_using_withcolumn_explode.where("match_id = 3825717").show(truncate=False)

# --> Numa partida um time pode ter mais de um técnico

print("Numa partida um time pode ter mais de um técnico")

+----------+------------------------------------------------------------------------------------------------------------+--------------------+-------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+--------------------------+----------------+----------+--------+------------+----------------+----------+-------------+---------------------------------------------+---------------+------------------------------------------------+--------------+---------+----------------------------------------------------------------------------------+
|away_score|away_team                                                                                                   |competition         |competition_stage  |home_score|home_team                                                                                                      

In [25]:
df_matches_bronze_manager_using_withcolumn_explode.count()

2894

In [26]:
df_matches_bronze_manager_using_select_explode = df_matches_bronze \
                            .select(F.explode_outer("home_team.managers").alias("manager_home"))

In [27]:
df_matches_bronze_manager_using_select_explode.count()

2894

## Competitions in Bronze Zone

In [31]:
path_competitions_bronze = 's3://bronze/competitions'

In [32]:
df_competition_bronze = spark.read.parquet(path_competitions_bronze)

In [33]:
df_competition_bronze.show(truncate=False)

+------------------+--------------+-------------------------+-----------------------+-----------------+-------------------------+--------------------------+--------------------------+--------------------------+--------------------------+-----------+---------+
|competition_gender|competition_id|competition_international|competition_name       |competition_youth|country_name             |match_available           |match_available_360       |match_updated             |match_updated_360         |season_name|season_id|
+------------------+--------------+-------------------------+-----------------------+-----------------+-------------------------+--------------------------+--------------------------+--------------------------+--------------------------+-----------+---------+
|male              |9             |false                    |1. Bundesliga          |false            |Germany                  |2023-12-12T07:43:33.436182|null                      |2023-12-12T07:43:33.436182|null      

## 

In [29]:
path_lineups_bronze = 's3://bronze/lineups'

